In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torchvision
from time import localtime, strftime
from torch.optim.lr_scheduler import StepLR

In [56]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model = nn.Sequential(*list(self.model.children())[:-1])  # Remove the last layer

    def forward(self, x):
        with torch.no_grad():
            features = self.model(x)
        return features.view(features.size(0), -1)
        
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class TeacherStudentNetworks:
    def __init__(self, feature_dim, output_dim):
        self.teacher = SimpleNN(feature_dim, output_dim)
        self.student = SimpleNN(feature_dim, output_dim)
        
class FeatureAugmenter:
    def __init__(self, input_dim):
        self.feature_augmenter = SimpleNN(input_dim, input_dim)
        
class BinaryClassifier:
    def __init__(self, input_dim):
        self.binary_classifier = SimpleNN(input_dim, 2)

class RealFakeDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label



In [39]:
def load_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []
    
    for model in os.listdir(root_dir):
        model_path = os.path.join(root_dir, model)
        if os.path.isdir(model_path):
            for label in ['0_real', '1_fake']:
                label_path = os.path.join(model_path, label)
                if os.path.isdir(label_path):
                    for img_name in os.listdir(label_path):
                        img_path = os.path.join(label_path, img_name)
                        image_paths.append(img_path)
                        labels.append(0 if '0_real' in label else 1)
                else:
                    for obj in os.listdir(model_path):
                        obj_path = os.path.join(model_path, obj)
                        label_path = os.path.join(obj_path, label)
                        if os.path.isdir(label_path):
                            for img_name in os.listdir(label_path):
                                img_path = os.path.join(label_path, img_name)
                                image_paths.append(img_path)
                                labels.append(0 if '0_real' in label else 1)
    
    return image_paths, labels

In [52]:
def train_teacher(teacher, feature_extractor, dataloader, device, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(teacher.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    feature_extractor.to(device)
    teacher.to(device)
    teacher.train()
    feature_extractor.eval()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in tqdm(dataloader, desc=f'Teacher training {epoch+1}/{num_epochs}', unit='batch'):
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            
            optimizer.zero_grad()
            outputs = teacher(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        scheduler.step()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
        save_checkpoint(teacher, filename='teacher_cpt', epoch=epoch)
        add_log(f'Saved teacher_cpt at epoch {epoch}. \nLoss: {running_loss/len(dataloader)}')
    print('Finished Training Teacher')

In [41]:
def train_student_augmenter(teacher, student, feature_extractor, augmenter, dataloader, device, num_epochs=10, margin=1.0):
    optimizer_student  = optim.Adam(student.parameters(), lr=0.001, weight_decay=0.0001)
    optimizer_augmenter = optim.Adam(augmenter.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler_student = StepLR(optimizer_student, step_size=5, gamma=0.5)
    scheduler_augmenter = StepLR(optimizer_augmenter, step_size=5, gamma=0.5)
    teacher.to(device)
    student.to(device)
    augmenter.to(device)
    feature_extractor.to(device)
    student.train()
    teacher.eval()
    feature_extractor.eval()
    augmenter.train()

    
    for epoch in range(num_epochs):

        running_loss_real = 0.0
        running_loss_fake = 0.0
        running_loss_augmenter = 0.0
        
        for images, labels in tqdm(dataloader, desc=f'Student and Augmenter training {epoch+1}/{num_epochs}', unit='batch'):
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            
            # Real images
            real_indices = (labels == 0)
            real_features = features[real_indices]
            if len(real_features) > 0:
                teacher_real = teacher(real_features)
                student_real = student(real_features)
                loss_real = F.mse_loss(teacher_real, student_real)
                
                optimizer_student.zero_grad()
                loss_real.backward()
                optimizer_student.step()
                
                running_loss_real += loss_real.item()
            
            # Fake images
            fake_indices = (labels == 1)
            fake_features = features[fake_indices]
            
            if len(fake_features) > 0:
                fake_indices = (labels == 1)
            fake_features = features[fake_indices]
            if fake_features.size(0) > 0:
                # Train the student network with fake features
                augmented_fake_features = augmenter(fake_features)
                teacher_fake = teacher(augmented_fake_features)
                student_fake = student(augmented_fake_features)
                discrepancy = torch.mean((teacher_fake / teacher_fake.norm(2, dim=1, keepdim=True) - student_fake / student_fake.norm(2, dim=1, keepdim=True)) ** 2)
                loss_fake = torch.clamp(margin - discrepancy, min=0.0)
                
                optimizer_student.zero_grad()
                loss_fake.backward()
                optimizer_student.step()
                
                running_loss_fake += loss_fake.item()

                # Train the augmenter network
                augmented_fake_features = augmenter(fake_features.detach())
                teacher_fake = teacher(augmented_fake_features)
                student_fake = student(augmented_fake_features.detach())
                loss_augmenter = F.mse_loss(teacher_fake, student_fake)
                
                optimizer_augmenter.zero_grad()
                loss_augmenter.backward()
                optimizer_augmenter.step()
                
                running_loss_augmenter += loss_augmenter.item()
        scheduler_student.step()
        scheduler_augmenter.step()
        print(f'Epoch {epoch+1}, Real Loss: {running_loss_real/len(dataloader)}, Fake Loss: {running_loss_fake/len(dataloader)}')
        save_checkpoint(student, filename='student_cpt', epoch=epoch)
        save_checkpoint(augmenter, filename='augmenter_cpt', epoch=epoch)
        add_log(f'Saved student_cpt and augmenter_cpt at epoch {epoch}. \nLoss student: real: {running_loss_real/len(dataloader)}, fake: {running_loss_fake/len(dataloader)}. \nLoss augmenter: {running_loss_augmenter/len(dataloader)}')
    print('Finished Training Student and Augmenter')

In [42]:
def train_student(teacher, student, feature_extractor, augmenter, dataloader, device, num_epochs=10, margin=1.0):
    optimizer = optim.Adam(student.parameters(), lr=0.001, weight_decay=0.0001)
    teacher.to(device)
    student.to(device)
    augmenter.to(device)
    feature_extractor.to(device)
    student.train()
    teacher.eval()
    feature_extractor.eval()
    augmenter.eval()
    
    for epoch in range(num_epochs):

        running_loss_real = 0.0
        running_loss_fake = 0.0
        
        for images, labels in tqdm(dataloader, desc=f'Student training {epoch+1}/{num_epochs}', unit='batch'):
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            
            # Real images
            real_indices = (labels == 0)
            real_features = features[real_indices]
            if len(real_features) > 0:
                teacher_real = teacher(real_features)
                student_real = student(real_features)
                loss_real = torch.mean((teacher_real - student_real) ** 2)
                
                optimizer.zero_grad()
                loss_real.backward()
                optimizer.step()
                
                running_loss_real += loss_real.item()
            
            # Fake images
            fake_indices = (labels == 1)
            fake_features = features[fake_indices]
            if len(fake_features) > 0:
                augmented_fake_features = augmenter(fake_features)
                teacher_fake = teacher(augmented_fake_features)
                student_fake = student(augmented_fake_features)
                discrepancy = torch.mean((teacher_fake / teacher_fake.norm(2, dim=1, keepdim=True) - student_fake / student_fake.norm(2, dim=1, keepdim=True)) ** 2)
                loss_fake = torch.clamp(margin - discrepancy, min=0.0)
                
                optimizer.zero_grad()
                loss_fake.backward()
                optimizer.step()
                
                running_loss_fake += loss_fake.item()
        
        print(f'Epoch {epoch+1}, Real Loss: {running_loss_real/len(dataloader)}, Fake Loss: {running_loss_fake/len(dataloader)}')
        save_checkpoint(student, filename='student_cpt', epoch=epoch)
        add_log(f'Saved student_cpt at epoch {epoch}. \nLoss student: real: {running_loss_real/len(dataloader)}, fake: {running_loss_fake/len(dataloader)}.')
    print('Finished Training Student')

In [43]:
def train_binary_classifier(teacher, student, classifier, feature_extractor, dataloader, device, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=0.001, weight_decay=0.0001)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    teacher.to(device)
    student.to(device)
    classifier.to(device)
    feature_extractor.to(device)
    classifier.train()
    teacher.eval()
    student.eval()
    feature_extractor.eval()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for images, labels in tqdm(dataloader, desc=f'Classifier training {epoch+1}/{num_epochs}', unit='batch'):
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            
            with torch.no_grad():
                teacher_outputs = teacher(features)
                student_outputs = student(features)
            discrepancies = (teacher_outputs - student_outputs) ** 2
            outputs = classifier(discrepancies)
            
            optimizer.zero_grad()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        scheduler.step()    
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
        save_checkpoint(classifier, filename='bin_classifier_cpt', epoch=epoch)
        add_log(f'Saved bin_classifier_cpt at epoch {epoch/len(dataloader)}')
    print('Finished Training Binary Classifier')

In [44]:
def train_augmenter(teacher, student, feature_extractor, augmenter, dataloader, device, num_epochs=10):
    optimizer = optim.Adam(augmenter.parameters(), lr=0.001, weight_decay=0.0001)
    teacher.to(device)
    student.to(device)
    augmenter.to(device)
    feature_extractor.to(device)
    teacher.eval()
    student.eval()
    feature_extractor.eval()
    augmenter.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for images, labels in tqdm(dataloader, desc=f'Augmenter training {epoch+1}/{num_epochs}', unit='batch'):
            images, labels = images.to(device), labels.to(device)
            features = feature_extractor(images)
            
            # Fake images
            fake_indices = (labels == 1)
            fake_features = features[fake_indices]
            if len(fake_features) > 0:
                augmented_fake_features = augmenter(fake_features)
                with torch.no_grad:
                    teacher_fake = teacher(augmented_fake_features)
                    student_fake = student(augmented_fake_features)
                loss = torch.mean((teacher_fake - student_fake) ** 2)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                running_loss+= loss.item()
                
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
        save_checkpoint(augmenter, filename='augmenter_cpt', epoch=epoch)
        add_log(f'Saved augmenter_cpt at epoch {epoch}. \nLoss: {running_loss/len(dataloader)}')
        
    print('Finished Training Augmenter')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])


In [46]:
def save_checkpoint(model, filename='checkpoint', epoch = 0):
    modelfilename = filename + f'_epoch{epoch}.pth.tar'
    torch.save(model.state_dict(), f'models/{modelfilename}')
    print(f'Saved checkpoint as: {modelfilename}')
    
def load_checkpoint(model, path):
    model.load_state_dict(torch.load(path))
    print(f'Loaded model: {path}')
    
def add_log(msg):
     with open('log.txt', 'a') as file:
        time = strftime("%H:%M", localtime())
        file.write(time + ': ' + msg + '\n')
         
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        torch.nn.init.constant_(m.bias, 0)

In [54]:
# Load image paths and labels
image_paths, labels = load_image_paths_and_labels('D:/CNN_synth/train_set/')

# Create a dataset instance 
full_dataset = RealFakeDataset(image_paths, labels, transform=transform)

# Define the split ratio
train_size = int(0.6 * len(full_dataset))
test_size = len(full_dataset) - train_size

# Split the dataset
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [57]:
len(train_dataset)

54197

In [49]:
# Initialize networks
feature_extractor = FeatureExtractor()
feature_dim = 2048  # ResNet50 output feature dimension
teacher_student = TeacherStudentNetworks(feature_dim, 2)
teacher_student.student.apply(init_weights)
teacher_student.teacher.apply(init_weights)
augmenter = FeatureAugmenter(feature_dim)
augmenter.feature_augmenter.apply(init_weights)
binary_classifier = BinaryClassifier(2)
binary_classifier.binary_classifier.apply(init_weights)

SimpleNN(
  (fc1): Linear(in_features=2, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_student.teacher.to(device)
teacher_student.student.to(device)
binary_classifier.binary_classifier.to(device)
feature_extractor.to(device)
augmenter.feature_augmenter.to(device)

SimpleNN(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2048, bias=True)
)

In [55]:
# Train the teacher network
train_teacher(teacher_student.teacher, feature_extractor, train_loader, device, 10)

Teacher training 1/10:   1%|▍                                                     | 12/1694 [00:14<32:43,  1.17s/batch]


KeyboardInterrupt: 

In [ ]:
# Train the student and the augmenter
train_student_augmenter(teacher_student.teacher, teacher_student.student, feature_extractor, augmenter.feature_augmenter, train_loader, device, 10)

In [ ]:
# Train the binary classifier
train_binary_classifier(teacher_student.teacher, teacher_student.student, binary_classifier.binary_classifier, feature_extractor, train_loader, device, 10)

testacc = test_binary_classifier(teacher_student.teacher, teacher_student.student, binary_classifier.binary_classifier, feature_extractor, test_loader, device)
mytestacc = test_binary_classifier(teacher_student.teacher, teacher_student.student, binary_classifier.binary_classifier, feature_extractor, mytest_loader, device)
add_log(f'Test dataset accuracy: {testacc}')
add_log(f'My test dataset accuracy: {mytestacc}')

In [ ]:

# for i in range(10):
#     # Train the student network
#     train_student(teacher_student.teacher, teacher_student.student, feature_extractor, augmenter, train_loader, device, 1, margin= 1.0)
#     save_checkpoint(teacher_student.teacher, filename='student_cpt', epoch=i)
    
#     # Traun the augmenter
#     train_augmenter(teacher_student.teacher, teacher_student.student, feature_extractor, augmenter, train_loader, device, 1)
#     save_checkpoint(teacher_student.teacher, filename='augmenter_cpt', epoch=i)
    

In [29]:
def test_binary_classifier(teacher, student, classifier, feature_extractor, dataloader, device):
    classifier.to(device)
    teacher.to(device) 
    student.to(device) 
    feature_extractor.to(device) 
    classifier.eval() # Set the classifier to evaluation mode
    teacher.eval()
    student.eval()
    feature_extractor.eval()

    all_labels = []
    all_preds = []
    
    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            # Extract features
            features = feature_extractor(images)
            
            # Get outputs from teacher and student networks
            teacher_outputs = teacher(features)
            student_outputs = student(features)
            
            # Calculate discrepancy
            discrepancy = (teacher_outputs - student_outputs) ** 2
            
            # Get predictions from binary classifier
            outputs = classifier(discrepancy)
            _, preds = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
    return accuracy

In [25]:
mytest_image_paths, mytest_labels = load_image_paths_and_labels('C:/Users/Danila/VSU/vsu_common_rep/vsu_common_rep/2year/2term/project/image_classification/content/mytestset/')
mytest_dataset = RealFakeDataset(mytest_image_paths, mytest_labels, transform=transform)
mytest_loader = DataLoader(mytest_dataset, batch_size=32, shuffle=False)

In [31]:
test_binary_classifier(teacher_student.teacher, teacher_student.student, binary_classifier.binary_classifier, feature_extractor, mytest_loader, device)

Test Accuracy: 52.95%


0.5294540229885057

In [27]:
print(len(mytest_dataset))

1392


In [229]:
limage_paths, mytest_labels = load_image_paths_and_labels("D:progan_train/progan_train/")
lfull_dataset = RealFakeDataset(mytest_image_paths, mytest_labels, transform=transform)
ltrain_size = int(0.7 * len(lfull_dataset))
ltest_size = len(lfull_dataset) - ltrain_size
ltrain_dataset, ltest_dataset = torch.utils.data.random_split(lfull_dataset, [ltrain_size, ltest_size])
ltest_loader = DataLoader(ltest_dataset, batch_size=32, shuffle=False)

In [105]:
save_checkpoint(teacher_student.teacher, filename='teacher_test_cpt', epoch=0)
save_checkpoint(teacher_student.student, filename='student_test_cpt', epoch=0)
save_checkpoint(augmenter.feature_augmenter, filename='augmenter_test_cpt', epoch=0)
save_checkpoint(binary_classifier.binary_classifier, filename='bin_classifier_test_cpt', epoch=0)

Saved checkpoint as: teacher_test_cpt_epoch0.pth.tar
Saved checkpoint as: student_test_cpt_epoch0.pth.tar
Saved checkpoint as: augmenter_test_cpt_epoch0.pth.tar
Saved checkpoint as: bin_classifier_test_cpt_epoch0.pth.tar


In [23]:
load_checkpoint(teacher_student.student, 'models_no_random_weights/student_cpt_epoch9.pth.tar')
load_checkpoint(teacher_student.teacher, 'models_no_random_weights/teacher_cpt_epoch9.pth.tar')
load_checkpoint(augmenter.feature_augmenter, 'models_no_random_weights/augmenter_cpt_epoch9.pth.tar')
load_checkpoint(binary_classifier.binary_classifier, 'models_no_random_weights/bin_classifier_cpt_epoch9.pth.tar')

Loaded model: models_no_random_weights/student_cpt_epoch9.pth.tar
Loaded model: models_no_random_weights/teacher_cpt_epoch9.pth.tar
Loaded model: models_no_random_weights/augmenter_cpt_epoch9.pth.tar
Loaded model: models_no_random_weights/bin_classifier_cpt_epoch9.pth.tar


In [35]:
train_augmenter(teacher_student.teacher, teacher_student.student, feature_extractor, augmenter, train_loader, device, 1)

Student training 1/1:   1%|▋                                                      | 26/2259 [00:10<14:41,  2.53batch/s]


KeyboardInterrupt: 